## Initialize Main Functions and Global Variables

In [1]:
#Import relevant Classes and make matplotlib inline to avoid loading plots at runtime
import geojson
import descarteslabs as dl
import descarteslabs.workflows as wf
import numpy as np
import math
import matplotlib.pyplot as plt
import config
%matplotlib inline
import functools
import pandas as pd
import pickle as pkl
import time
from gzip import compress, decompress
from ipyleaflet import GeoJSON, Popup
from ipywidgets import HTML, Button, VBox, HBox, Box, Layout, Output, Text, Label
from descarteslabs import Storage, Auth
import random

/opt/conda/lib/python3.7/site-packages/descarteslabs/_dl_modules/common/workflows/arrow_serialization/context.py:57: FutureWarning: 'pyarrow.SerializationContext' is deprecated and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  serialization_context = pa.SerializationContext()
/opt/conda/lib/python3.7/site-packages/descarteslabs/_dl_modules/common/workflows/arrow_serialization/context.py:58: FutureWarning: 'pyarrow.register_default_serialization_handlers' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  pa.register_default_serialization_handlers(serialization_context)


In [2]:
#Set a number of global parameters for the entire form, edit this to specifications
#Number of images to shown at one time
IMGSHOWN=30
#Number of tile pairs to show per row
NUMIMGROW=3
#Name of the current dataset, change as needed
datasetName1='high_sampledworried_puppy'
datasetName2='updatedModel_state_sampled_slender_gerbil'
datasetName3='updatedModel_state_sampled_happy_cow'

In [3]:
"""
Initiate the loadGroup function which takes the image dicts, group dict and the map application to create and load the entire set of images and geojsons (GJS) in an organized fashion using
IPyWidgets.  The orientation of the UI is set but the number of images shown at one time and the number of images per row shown can be altered to desire.  Note that doing so can overload the
kernel or make the images too small to effectively observe any real detail.  The complete ordered lists are then returned to the caller.

INPUT
    randomKeyImageDict->
        dict{
            ...,
            string():numpy.ndarray[],
            string():numpy.ndarray[],
            ...
        }
        Exactly the same as the LrBatch dictionary uploaded to the storage in Grouping.ipynb
    randomKeyHrImgDict->
        dict{
            ...,
            string():numpy.ndarray[],
            string():numpy.ndarray[],
            ...
        }
        Exactly the same as the HrBatch dictionary uploaded to the storage in Grouping.ipynb
    groups->
         dict{
            ...,
            string(int()):list[...,string(),string(),string()...],
            string(int()):list[...,string(),string(),string()...],
            ...
        }
        Exactly the same as the GroupBatch dictionary uploaded to the storage in Grouping.ipynb
    m->
        wf.interactive.MapApp()
        Previously initialized map application to tie the async buttons to the future UI
    userNameValue->
        string()
        The inputted userName from the map application, refer below for more information
    confirmed->
        dict{
            ...,
            string():dict{
                string():time(),
                string():string()
            },
            ...
        }
        A dictionary containing the keys of user confirmed tiles as keys and a dictionary, containing the time at which the confirmation was made and the user's name as values, as values.  This
        is used to reference and update the confirmed lists as the user interacts with the web app
    denied->
        dict{
            ...,
            string():dict{
                string():time(),
                string():string()
            },
            ...
        }
        Same as the above confirmed dictionary but filled with user denied tile keys
RETURNS
    groupsGJS->
        list[
            ...,
            list[
                  ...,
                  list[
                      ...,
                      GeoJSON(),
                      ...
                  ],
                  ...
            ],
            ...
        ]
        This is the group-batch ordered list of GeoJSONs which can then be iteratively used to add to the MappApp.  The first layer represents the group clusters, the second layer represents
        the batch of IMGSHOWN sized list with individual GeoJSONs representing the repective key as values in the list.
    groupsImg->
        list[
            ...,
            list[VBox(list[
                ...,
                HBox(list[
                    ...,
                    VBox(list[HBox(list[
                            Output(),
                            OutPut()
                        ]),HBox(list[
                            Button(),
                            Button(),
                            Button()
                        ])
                    ]),
                    ...
                ]),
                ...
            ])],
            ...
        ]
        This is the group-batch ordered list of Images which can then be iteratively used to add the image UI to the 'refreshable' Output of the final UI. The first layer represents the group 
        clusters, the second layer represents the batch of IMGSHOWN sized Vertically aligned UI, the third layer represents the NUMIMGROW sized group, Horizontally aligned UI, with the fourth
        layer representing the Vertically aligned UI of pair of images and three buttons, and the final layer representing the individual Output/Button IPyWidget widgets that respectively
        represent an image or a button. For a visual representation of the third layer and below refer to the diagram below.
              _____________________________________________________________________________________________________________________________
        VBOX-|      _____________________________________________________________________________________________________________________ |
             |HBOX-|      ________________________________       ________________________________       ________________________________ ||
             |     |VBOX-|      _________________________ |VBOX-|      _________________________ |VBOX-|      _________________________ |||
             |     |     |HBOX-|_____Output1,Output2_____||     |HBOX-|_____Output1,Output2_____||     |HBOX-|_____Output1,Output2_____||||
             |     |     |      _________________________ |     |      _________________________ |     |      _________________________ |||
             |     |     |HBOX-|_Button1,Button2,Button3_||     |HBOX-|_Button1,Button2,Button3_||     |HBOX-|_Button1,Button2,Button3_||||
             |     |     |________________________________|     |________________________________|     |________________________________|||
             |     |_____________________________________________________________________________________________________________________||
             |      _____________________________________________________________________________________________________________________ |
             |HBOX-|      ________________________________       ________________________________       _________________________________||
             |     |VBOX-|      _________________________ |VBOX-|      _________________________ |VBOX-|      _________________________ |||
             |     |     |HBOX-|_____Output1,Output2_____||     |HBOX-|_____Output1,Output2_____||     |HBOX-|_____Output1,Output2_____||||
             |     |     |      _________________________||     |      _________________________ |     |      _________________________ |||
             |     |     |HBOX-|_Button1,Button2,Button3_||     |HBOX-|_Button1,Button2,Button3_||     |HBOX-|_Button1,Button2,Button3_||||
             |     |     |________________________________|     |________________________________|     |________________________________|||
             |     |_____________________________________________________________________________________________________________________||
             |____________________________________________________________________________________________________________________________|
        *The number of items in the second layer HBOX is dependent on NUMIMGROW
        *The number of rows in the first layer VBOX is dependent on math.ceil(IMGSHOW/NUMIMGROW)  
WORKFLOW
   -Initialize the Button on click events for each individual image pair, these will later be called and tied to actual buttons
      -confirmKiln->
          INPUT
              b->
                  Button()
                  A self-referential Button item, refers to whichever confirm button was clicked
              other->
                  Button()
                  The other paired deny Button with the image pairs
          RETURNS
              None
          WORKFLOW
             -Check if there exists a key in the inherited denied list with the same name, if so remove it and reset the button to a 'gray' color
             -Check if there exists a key in the inherited confirmed list with the same name, if not add it with the current time and userName as referential values
             -Make the button green to signify the image has been added to the confirmed list
      -denyKiln->
          INPUT
              b->
                  Button()
                  A self-referential Button item, refers to whichever deny button was clicked
              other->
                  Button()
                  The other paired confirm Button with the image pairs
          RETURNS
              None
          WORKFLOW
             -Check if there exists a key in the inherited confirmed list with the same name, if so remove it and reset the button to a 'gray' color
             -Check if there exists a key in the inherited denied list with the same name, if not add it with the current time and userName as referential values
             -Make the button red to signify the image has been added to the denied list
       -centerImage()->
          INPUT
              b->
                  Button()
                  A self-referential Button item, refers to whichever confirm button was clicked
          RETURNS
              None
          WORKFLOW
             -Center attention to the image clicked on the MapApp by calling it's dltile and using it's centroid feature
   -Initialize empty lists which will contain the ordered image and GJS lists, refer to RETURN
   -Iterate through the groups keys in sequential order and for each initialize the temporary buffer lists which will fill the return arrays, refer to RETURN
      -Iterate through each key in the specified group and then fill the respective lists with the subsequent ordering
         -Check if the limit of IMGSHOWN has been reached, add the current results to the batches lists and reset the limit and temp lists
         -Query for the GJS and add it to the GJS batch list 
         -Check if the current row has been filled with NUMIMGROW images and if so add to the batch list and reset the temp lists
         -Initialize the confirm, deny and center buttons with the key tied to it's tooltip feature and each button tied to it's respective function, look above for more details.
         -Create an output for the high and low res image and within it create a separate figure to display the respective image
         -Tie the entire cluster of widgets together using the requisite layout and continue to iterate, for list architecture and visual layout refer to the RETURN
   -Return the finalized ordered lists for use
KNOWN_ISSUES
   -The gray color used does not match the original button, I haven't been able to figure out what color it is and frankly don't really care too much, OCD people beware and I'm sorry
   -Since we're pre-loading all of the UI, this function takes a large while to finish running so set aside enough time beforehand to allow the loading of the individual batched UI and for any
    unforseen issues
   -I am not overly familiar with UI/UX design or IPyWidgets overall which makes this design a bit... 'janky'.  But it does work as intended with mostly minor inconveniences, if you have more
    knowledge or can think of different better alternatives, feel free to do so.
   -Currently I have found no 'effective' way of stably using the built-in Image widget with descarteslabs arrays and am instead using individual matplotlib figures to display the individual
    images with the Output widget capturing the displayed output.  This is not exactly ideal since matplotlib is rather slow and loading 2500 (currently) images at once is both time and memory
    intensive.  I have been unable to find a workaround but using Output to capture some other diplay may be effective in speeding the process up.
   -Buttons are initialized beforehand, so if a user were to load the same group-batch, alter the confirmed/denied list and save it mid run, the current user would not see those changes
    reflected on the button colors, but would still be able to change and submit their labelings
"""

"\nInitiate the loadGroup function which takes the image dicts, group dict and the map application to create and load the entire set of images and geojsons (GJS) in an organized fashion using\nIPyWidgets.  The orientation of the UI is set but the number of images shown at one time and the number of images per row shown can be altered to desire.  Note that doing so can overload the\nkernel or make the images too small to effectively observe any real detail.  The complete ordered lists are then returned to the caller.\n\nINPUT\n    randomKeyImageDict->\n        dict{\n            ...,\n            string():numpy.ndarray[],\n            string():numpy.ndarray[],\n            ...\n        }\n        Exactly the same as the LrBatch dictionary uploaded to the storage in Grouping.ipynb\n    randomKeyHrImgDict->\n        dict{\n            ...,\n            string():numpy.ndarray[],\n            string():numpy.ndarray[],\n            ...\n        }\n        Exactly the same as the HrBatch dict

In [4]:
def loadGroup(randomKeyImageDict,randomKeyHrImgDict,groups,m,userNameValue,confirmed,denied,edit,add):
    #Create Button Click Events
    def confirmKiln(other,b):
        if other.style.button_color == 'red':
            other.style.button_color='gray'
        if (b.tooltip in denied.value.keys()):
            edit.value[b.tooltip]={'add':'Confirmed','data':{"time":time.time(),"user":userNameValue.value}}
        elif b.tooltip in edit.value:
            edit.value[b.tooltip]={'add':'Confirmed','data':{"time":time.time(),"user":userNameValue.value}}
        elif (b.tooltip not in confirmed.value.keys()):
            add.value[b.tooltip]={'add':'Confirmed','data':{"time":time.time(),"user":userNameValue.value}}
        b.style.button_color='green'
    def denyKiln(other,b):
        if other.style.button_color == 'green':
            other.style.button_color='gray'
        if (b.tooltip in confirmed.value.keys()):
            edit.value[b.tooltip]={'add':'Denied','data':{"time":time.time(),"user":userNameValue.value}}
        elif (b.tooltip in edit.value):
            edit.value[b.tooltip]={'add':'Denied','data':{"time":time.time(),"user":userNameValue.value}}
        elif (b.tooltip not in denied.value.keys()):
            add.value[b.tooltip]={'add':'Denied','data':{"time":time.time(),"user":userNameValue.value}}
        b.style.button_color='red'
    def centerImage(b):
        m.center=(dl.scenes.DLTile.from_key(b.tooltip).geometry.centroid.xy[1][0],dl.scenes.DLTile.from_key(b.tooltip).geometry.centroid.xy[0][0])
    
    #Initialize return GJS and main Image display lists
    groupsGJS=[]
    groupsImg=[]
    
    #Iterate through the entire groups list and add the images and gjs to their respective position
    for group in range(len(groups.keys())):
        print("Adding group #",group," to the web app")
        #Initialize/Reset all relevant lists
        batchesGJS=[]
        batchGJS=[]
        batchesImg=[]
        batchImg=[]
        rowImg=[]
        limit=0
        for key in groups[str(group)]:
            #Limit reached, add the current batch images and gjs to the respective batch list and reset for next batch
            if limit == IMGSHOWN:
                batchImg.append(HBox(rowImg))
                batchesImg.append(VBox(batchImg))
                batchesGJS.append(batchGJS)
                batchGJS=[]
                batchImg=[]
                rowImg=[]
                limit=0
            
            #Load GJS
            subtilegjs=GeoJSON(
                data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
                style={"color":"red",
                      "fillOpacity":0}
            )

            batchGJS.append(subtilegjs)
            
            #Add and reset row if row is equal to NUMROWIMG
            if (len(rowImg) >= NUMIMGROW):
                batchImg.append(HBox(rowImg))
                rowImg=[]
                
            #Initialize Buttons and tie their respective click events
            confirmButton = Button(
                description='Confirm Kiln',
                tooltip=key
            )

            denyButton = Button (
                description="No Kiln",
                tooltip=key
            )
            
            if key in confirmed.value.keys():
                confirmButton.style.button_color='green'
            if key in denied.value.keys():
                denyButton.style.button_color='red'
            centerButton = Button (
                description="Center to Image",
                tooltip=key
            )

            confirmButton.on_click(functools.partial(confirmKiln,denyButton))
            denyButton.on_click(functools.partial(denyKiln,confirmButton))
            centerButton.on_click(centerImage)    
            
            #Create Output for the matplotlib plots and tie them to the respective image plot
            output=Output(layout={'border': '1px solid black'})
            output2=Output(layout={'border':'1px solid black'})
            with output:
                fig = plt.figure(figsize=(6,6))
                hr=plt.imshow(np.swapaxes(randomKeyHrImgDict[key],0,2))
                plt.show(fig)
            with output2:
                fig2=plt.figure(figsize=(6,6))
                lr=plt.imshow(np.swapaxes(randomKeyImageDict[key],0,2))
                plt.show(fig2)
            
            #Consolidate and order the Images and Buttons into a VBox and increase the limit
            rowImg.append(VBox([HBox([output,output2]),HBox([confirmButton,denyButton,centerButton])]))
            limit+=1
            
        #Do one final append to add any straggling gjs and images and add the total batch set to the respective group list
        batchesGJS.append(batchGJS)
        groupsGJS.append(batchesGJS)
        batchImg.append(HBox(rowImg))
        batchesImg.append(VBox(batchImg))
        groupsImg.append(batchesImg)
    return groupsGJS,groupsImg

In [5]:
"""
This is the main callable function that initializes the static parts of the UI and preloads any data or lists that will be required by the more intensive loadGroup call.  This function returns
the displayable UI for the given batch which the user will interact with, this function will be called by the user for each batch they want/need loaded.

INPUTS
    confirmed->
        dict{
            ...,
            string():dict{
                string():time(),
                string():string()
            },
            ...
        }
        A dictionary containing the keys of user confirmed tiles as keys and a dictionary, containing the time at which the confirmation was made and the user's name as values, as values.  This
        is used to reference and update the confirmed lists as the user interacts with the web app
    denied->
        dict{
            ...,
            string():dict{
                string():time(),
                string():string()
            },
            ...
        }
        Same as the above confirmed dictionary but filled with user denied tile keys
    groups->
         dict{
            ...,
            string(int()):list[...,string(),string(),string()...],
            string(int()):list[...,string(),string(),string()...],
            ...
        }
        Exactly the same as the GroupBatch dictionary uploaded to the storage in Grouping.ipynb
    batch->
        int()
        An integer representing the current batch of images, used to access the descarteslabs storage uploaded dictionaries, ensure this number is a valid value in your storage.
RETURN
    total->
        VBox(list[
            Vbox(list[
                Box(list[
                    MapApp()
                ]),
                ***Output()***
            ]), HBox(list[
                Button()
            ])
        ])
        This is the static parts of the UI that are initially generated and returned to the user to display.  Everything within this UI is run asynchronously and allows for user input to
        proceed.  The first layer represents the vertically aligned map-image combo and the nextGroup Button widget, the next layer within the map-image combo separates the MapApp and the
        ***Output*** vertically.  The ***Output*** widget is a static representation that is iteratively cleared and refreshed with the next UI from the groupImg batch-group, for more
        information and a visual represenation of it refer to the RETURNS section of the loadGroup fucntion above.  For a visual representation of this display refer to the diagram below.
              _________________________________________________
        VBOX-|      __________________________________________ |
             |VBOX-|     ___________________________________  ||
             |     |BOX-|_____________MapApp________________| ||
             |     |_______________***Output***_______________||
             |      _________  _________  _________  _________ |
             |HBOX-|_Button1_||_Button2_||_TextBox_||_Button3_||
             |_________________________________________________|
WORKFLOW
   -Clear the matplotlib from previous uses to scrap and save memory, otherwise plots persist between loads
   -Query the descarteslabs storage for the image dictionaries associated with the given datasetName and batch number
   -Initialize MapApp with no parameters
   -Initialize the TextBox class object to keep changes made to the userName asynchronously
   -Use loadGroup to load the ***Output*** UI for each group-batch and the group-batch GJS to update the MapApp asynchronously.  For more information, refer to the above information on the
    function
   -Create the nextGroup, saveData, and Submit buttons for the final UI alongside the userName textBox and the label displaying the user and the current batch and group displayed.
   -Create a Counter class and initialize two counters, one for batches and another for groups to be able to asynchronously keep track of the current values for each as we load different
    sections of the dataset.
   -Create the saveDataFunc on-click event for the saveData Button and tie them together.  This button is responsible for taking the current confirmed and denied dictionaries and merging them
    with a freshly queried version of the same currently stored in the storage under datasetName+'Confirmed'/'Denied'.  Overwrite the current confirmed and denied dictionaries with the merged
    versions and return.
   -Create the submitNameFunc on-click event for the submit button and tie them together.  This button checks the current userNameValue and ensures the submitted userName is not empty or
    contains non alphabetic characters.  If it doesn't the userNameValue is updated and the label is changed to reflect the current userName as well as the buttom turning green.  If it fails
    the check the button turns red and a warning is printed to the log.
   -Create the loadNext on-click event and tie it to the previously initialized button, this button is responsible for asynchronously swapping and refreshing the relevant ***Output*** and
    map information
       -Ensure that we aren't overstepping our group size and ignore any further calls once we do
       -Clear the map of previous GJS
       -Check if the current batch is greater than the total number of batches for the current group, and if so, move onto the next group and reset the batch counter
       -Use the group and batch values to refer to the current groupGJS and groupImg lists/UI respectively
          -For the GJS, iteratively add the each from the batch to the map
          -For the ImageUI, obtain the current UI and display it using the HiImg ***Output*** after the ***Output*** is cleared, effectively 'refreshing' that section of the UI with new info
       -Increment the batch counter and await the next click
   -Create the final UI elements and adjust them as required before returning the final UI object back to the caller.  For more information on the UI, refer to the RETURN section.
KNOWN_ISSUES
   -The most major issue with this is the inability to effectively stop labeling without redoing a portion of the data.  Take for example a set of 1000 images, with 3 groups and 3 batches per
    group.  As it stands, a user can label the first entire group and separately save the confirmed and denied lists to storage, but if the user quits, the kernel dies or some unforseen quit
    call is made the user will need to re-run on the current parameters (datasetName and batch) and click until the first group is past before proceeding.  Additionally, the user will need to
    update, not set, any uploaded confirmed/denied lists.  Some debugging information has been added to the log but otherwise that functionality has not been added due to time and scope
    constraints.
   -The next big issue is a concurrency race condition that may arise if two users saveData at around the same time.  Since the on-click function calls the storage, and edits it before saving
    it there is a brief window during which another user can do the same before the previous data is saved such that only the newest data is kept and the other data is lost.  In theory this
    isn't likely given how brief the window for it currently is and additionally may not actively be a problem if the users don't terminate the program before saving again as their 'lost' data
    is maintained in their personal session and can be submitted with no issues later on.
   -A maybe issue, maybe not is that there is a limit to the size of any storage item, and since we're only keeping one confirmed and denied object in storage, if the dataset is large enough
    there is a real possibility of reaching this limit without intending to.  As it stands I don't believe the issue is relevant with 20000 images as the things stored are basically strings,
    but it may be necessary in the future to handle this potentiality.
   -Besides that, most of the KNOWN_ISSUES from the loadGroup section apply doubly here and any adjustments should be made in conjunction in the hopes of improving the process. 
"""

"\nThis is the main callable function that initializes the static parts of the UI and preloads any data or lists that will be required by the more intensive loadGroup call.  This function returns\nthe displayable UI for the given batch which the user will interact with, this function will be called by the user for each batch they want/need loaded.\n\nINPUTS\n    confirmed->\n        dict{\n            ...,\n            string():dict{\n                string():time(),\n                string():string()\n            },\n            ...\n        }\n        A dictionary containing the keys of user confirmed tiles as keys and a dictionary, containing the time at which the confirmation was made and the user's name as values, as values.  This\n        is used to reference and update the confirmed lists as the user interacts with the web app\n    denied->\n        dict{\n            ...,\n            string():dict{\n                string():time(),\n                string():string()\n         

In [6]:
def InitiateUI(confirmed,denied,groups,batch,datasetName):
    #Clear existing plts
    plt.clf()
    
    #load image data from storage
    print("Loading Data")
    randomKeyImageDict=pkl.loads(decompress(storage.get(datasetName+"LrBatch"+str(batch))))
    randomKeyHrImgDict=pkl.loads(decompress(storage.get(datasetName+"HrBatch"+str(batch))))
      
    #initialize Map
    m = wf.interactive.MapApp()
    
    class TextBox:
        def __init__(self,initial='noUser'):
            self.value=initial
        def changeName(self,text):
            self.value=text
            return;
    userNameValue=TextBox()
    class updateList:
        def __init__(self,initial=[]):
            self.value=initial
            
    confirmedClass=updateList(confirmed)
    deniedClass=updateList(denied)
    class editRequest:
        def __init__(self):
            self.value={}
    class addRequest:
        def __init__(self):
            self.value={}        
    edit=editRequest()
    add=addRequest()
    
    #iterate through each cluster group and pre-load all the UI and return the lists
    print("Initializing UI!")
    groupGJS,groupImg=loadGroup(randomKeyImageDict,randomKeyHrImgDict,groups,m,userNameValue,confirmedClass,deniedClass,edit,add)
    print('Done!')
    
    #Initialize counter class and two counters to iteratively keep track between on-click even
        
    class Counter:
       def __init__(self, initial=0):
          self.value = initial

       def increment(self, amount=1):
          self.value += amount
          return self.value

       def reset(self):
            self.value=0

       def __iter__(self, sentinal=False):
          return iter(self.increment, sentinal)

    groupcounter=Counter()
    batchcounter=Counter()
    
    #create buttons and textBox to insert to our layout
    nextGroup=Button(
        description="Next Group",
        tooltip="Proceed"
    )
    
    saveData=Button(
        description="Save Data",
        tooltip="Save"
    )
    
    userName=Text(
        placeholder='Type your name!',
        description='User Name',
        disabled=False
    )
    
    submitName=Button(
        description="Submit",
        tooltip="Submit"
    )
    

    curLabel=Label(value="User: "+userNameValue.value+", Working on: Group: "+str(groupcounter.value)+", Batch: "+str(batchcounter.value))

    
    #On-Click call the next batch of gjs and images or the next group if the batches are done for that group
    def saveDataFunc(confirmed,denied,add,edit,batchNum,b):
        confirmedName='JSErrorConfirmed'
        deniedName='JSErrorDenied'
        
        newConfirmed=pkl.loads(decompress(storage.get(datasetName+confirmedName)))
        newDenied=pkl.loads(decompress(storage.get(datasetName+deniedName)))
        for req in add.value:
            if add.value[req]['add']=='Confirmed':
                newConfirmed[req]=add.value[req]['data']
            elif add.value[req]['add']=='Denied':
                newDenied[req]=add.value[req]['data']
        for req in edit.value:
            if edit.value[req]['add']=='Confirmed':
                newConfirmed[req]=edit.value[req]['data']
                if req in newDenied:
                    del(newDenied[req])
            elif edit.value[req]['add']=='Denied':
                newDenied[req]=edit.value[req]['data']
                if req in newConfirmed:
                    del(newConfirmed[req])
        storage.set(datasetName+confirmedName,compress(pkl.dumps(newConfirmed)))
        storage.set(datasetName+deniedName,compress(pkl.dumps(newDenied)))
        confirmed.value=newConfirmed
        denied.value=newDenied
        add.value={}
        edit.value={}
        
        
    def submitNameFunc(b,userNameValue=userNameValue,groupcounter=groupcounter,batchcounter=batchcounter):
        bad='1234567890-=[]\\;\',./!@#$%^&*()_+{}|:\"<>? '
        if userName.value and not any(x in userName.value for x in bad):
            userNameValue.changeName(userName.value)
            curLabel.value="User: "+userNameValue.value+", Working on: Group: "+str(groupcounter.value)+", Batch: "+str(batchcounter.value)
            b.style.button_color='green'
        else:
            print('Empty input or not alphabetic found in input')
            b.style.button_color='red'
    
    def loadNextGroup(b,groupcounter=groupcounter,batchcounter=batchcounter):
        if (groupcounter.value < len(groups.keys())):
            #Reset map
            m.clear_layers()
            
            #Check if Batch # is larger than it should be and move onto the next group
            if (batchcounter.value >= math.ceil(len(groups[str(groupcounter.value)])/IMGSHOWN)):
                groupcounter.increment()
                batchcounter.reset()
            
            curLabel.value="User: "+userNameValue.value+", Working on: Group: "+str(groupcounter.value)+", Batch: "+str(batchcounter.value)
            curBatch.clear_output()
            with curBatch:
                display(curLabel)
                
            #Print current grouping information for debugging/early termination purposes
            print('group #')
            print(groupcounter.value)
            print('batch #')
            print(batchcounter.value)
            print('gjs group batch #')
            print(len(groupGJS[groupcounter.value]))
            print('group size')
            print(len(groupImg[groupcounter.value]))
            
            #Add the relevant group-batch gjs to the map
            for gjs in groupGJS[groupcounter.value][batchcounter.value]:
                m.add_layer(gjs)
            
            #Initialize the group-batch images from the list and display it using a clearable output to effectively 'refresh'
            horizontalBox=VBox([groupImg[groupcounter.value][batchcounter.value]])
            
            HiImg.clear_output()
            with HiImg:
                display(horizontalBox)
            
            #Increase step counter!
            batchcounter.increment()
            
    #Tie next group button to it's on-click event
    saveData.on_click(functools.partial(saveDataFunc,confirmedClass,deniedClass,add,edit,batch))
    nextGroup.on_click(loadNextGroup)
    submitName.on_click(submitNameFunc)

    #create custom box layout for the future image section, map and buttons
    HiImg=Output(layout={'border': '1px solid black'})
    horizontalBox=Box([HiImg],layout=Layout(
        display='flex',
        flex_flow='row',
        width='100%'
    ))
    curBatch=Output(layout={'border': '1px solid black'})
    with curBatch:
        display(curLabel)
    horizontalBox1=Box([m,curBatch],layout=Layout(
        display='flex',
        flex_flow='column',
        width='100%'
    ))
    
    #create actual layout for the UI
    maps=VBox([horizontalBox1,horizontalBox])
    button=HBox([nextGroup,saveData,userName,submitName])
    total=VBox([maps,button])
    
    #Return the callable UI!
    return total


## Look over these next cells and edit/change or initialize any necessary values
# Choose a valid number for JLBatch and JSBatch
# RUN A SINGLE UI PER KERNEL LOAD, ONCE ONE IS DONE, RESTART THE KERNEL AND CONTINUE WITH THE NEXT!

In [7]:
#Initialize the basic variables
storage=Storage(auth=Auth(client_id=config.ID,client_secret=config.SECRET))
#Juan's Data, can be any 0-2!
JSbatch=0
print('Working on ',JSbatch)

Working on  0


## This is Juan's Labeled Dataset!

In [ ]:
print("Juan's Batch #",JSbatch)
if datasetName1+'JSErrorConfirmed' not in storage.list():
    storage.set(datasetName1+'JSErrorConfirmed',compress(pkl.dumps({})))
    JSconfirmed={}
else:
    JSconfirmed=pkl.loads(decompress(storage.get(datasetName1+'JSErrorConfirmed')))
if datasetName1+'JSErrorDenied' not in storage.list():
    storage.set(datasetName1+'JSErrorDenied',compress(pkl.dumps({})))
    JSdenied={}
else:    
    JSdenied=pkl.loads(decompress(storage.get(datasetName1+'JSErrorDenied')))
JSUI=InitiateUI(JSconfirmed,JSdenied,pkl.loads(decompress(storage.get(datasetName1+'GroupBatch'+str(JSbatch)))),JSbatch,datasetName1)

In [ ]:
#Call the result to display in the below cell, continue until images no longer load.
JSUI

## Create separate datasets for Juan and Jihyeon from their labeled data

In [8]:
#Get separate Batch datasets for Juan and Jihyeon!
Combinedconfirmed=pkl.loads(decompress(storage.get(datasetName1+'Confirmed'))).keys()
Combineddenied=pkl.loads(decompress(storage.get(datasetName1+'Denied'))).keys()
JuanBatch=pkl.loads(decompress(storage.get(datasetName1+'GroupBatch'+str(JSbatch))))
JuanBatchKeys=[]
for x in JuanBatch:
    JuanBatchKeys.extend(JuanBatch[x])

JuanConfirmed={key for key in JuanBatchKeys if key in Combinedconfirmed}
JuanDenied={key for key in JuanBatchKeys if key in Combineddenied}

In [9]:
#Print number to ensure all data has been loaded and separated
print(len(JuanConfirmed)+len(JuanDenied))
print(len(JuanBatchKeys))

2095
2095


## Calculate the Error Rates for Juan's and Jihyeon's batch using the finished confirmed and denied lists 

In [10]:
JSconfirmed=pkl.loads(decompress(storage.get(datasetName1+'JSErrorConfirmed')))
JSdenied=pkl.loads(decompress(storage.get(datasetName1+'JSErrorDenied')))

In [11]:
#Juan's Error Analysis
fplist=[key for key in JuanConfirmed if key not in JSconfirmed]
tplist=[key for key in JuanConfirmed if key in JSconfirmed]
fnlist=[key for key in JuanDenied if key not in JSdenied]
tnlist=[key for key in JuanDenied if key in JSdenied]
fp=len(fplist)/(len(fplist)+len(tnlist))
tp=len(tplist)/(len(tplist)+len(fnlist))
fn=len(fnlist)/(len(fnlist)+len(tplist))
tn=len(tnlist)/(len(tnlist)+len(fplist))
p=len(tplist)/(len(tplist)+len(fplist))
r=len(tplist)/(len(tplist)+len(fnlist))
print('Error rates for Juan\'s Batch',JSbatch)
print('False Positive:',fp)
print('True Positive:',tp)
print('False Negative:',fn)
print('True Negative:',tn)
print('Precision:',p)
print('Recall:',r)

Error rates for Juan's Batch 0
False Positive: 0.01510912143256855
True Positive: 0.8961038961038961
False Negative: 0.1038961038961039
True Negative: 0.9848908785674314
Precision: 0.9108910891089109
Recall: 0.8961038961038961


In [ ]:
JSMap=wf.interactive.MapApp()
JSMap.clear_layers()
JSMap.center=(22.3094, 83.4160)
JSMap.zoom=6
def showPopup(event,feature,properties):
    JSMap.add_layer(Popup(location=(dl.scenes.DLTile.from_key(properties['style']['key']).geometry.centroid.xy[1][0],dl.scenes.DLTile.from_key(properties['style']['key']).geometry.centroid.xy[0][0]),child=HTML(value='Key: '+properties['style']['key']+'<br>Description: '+properties['style']['description']),close_button=True))

In [ ]:
#FALSE POSITIVES
for key in fplist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"red",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'False Positive'},
        hover_style={'color':'blue'},
    )
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)

In [ ]:
JSMap.clear_layers()
#FALSE NEGATIVES
for key in fnlist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"orange",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'False Negative'},
        hover_style={'color':'blue'},
    )
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)

In [ ]:
JSMap.clear_layers()
#TRUE POSITIVE
for key in tplist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"green",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'True Positive'},
        hover_style={'color':'blue'},
    )    
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)    

In [ ]:
JSMap.clear_layers()
for key in tnlist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"yellow",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'True Negative'},
        hover_style={'color':'blue'},
    )    
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)

In [ ]:
JSMap

## Next Dataset!

In [12]:
#Initialize the basic variables
storage=Storage(auth=Auth(client_id=config.ID,client_secret=config.SECRET))

#Juan's Data, can be any 0-1!
JSbatch=0
print('Working on ',JSbatch)

Working on  0


## This is Juan's Labeled Dataset!

In [ ]:
print("Juan's Batch #",JSbatch)
if datasetName2+'JSErrorConfirmed' not in storage.list():
    storage.set(datasetName2+'JSErrorConfirmed',compress(pkl.dumps({})))
    JSconfirmed={}
else:
    JSconfirmed=pkl.loads(decompress(storage.get(datasetName2+'JSErrorConfirmed')))
if datasetName2+'JSErrorDenied' not in storage.list():
    storage.set(datasetName2+'JSErrorDenied',compress(pkl.dumps({})))
    JSdenied={}
else:    
    JSdenied=pkl.loads(decompress(storage.get(datasetName2+'JSErrorDenied')))
JSUI=InitiateUI(JSconfirmed,JSdenied,pkl.loads(decompress(storage.get(datasetName2+'GroupBatch'+str(JSbatch)))),JSbatch,datasetName2)

In [ ]:
#Call the result to display in the below cell, continue until images no longer load.
JSUI

## Create separate datasets for Juan and Jihyeon from their labeled data

In [13]:
#Get separate Batch datasets for Juan and Jihyeon!
Combinedconfirmed=pkl.loads(decompress(storage.get(datasetName2+'Confirmed'))).keys()
Combineddenied=pkl.loads(decompress(storage.get(datasetName2+'Denied'))).keys()
JuanBatch=pkl.loads(decompress(storage.get(datasetName2+'GroupBatch'+str(JSbatch))))
JuanBatchKeys=[]
for x in JuanBatch:
    JuanBatchKeys.extend(JuanBatch[x])

JuanConfirmed={key for key in JuanBatchKeys if key in Combinedconfirmed}
JuanDenied={key for key in JuanBatchKeys if key in Combineddenied}

In [14]:
#Print number to ensure all data has been loaded and separated
print(len(JuanConfirmed)+len(JuanDenied))
print(len(JuanBatchKeys))

2081
2081


## Calculate the Error Rates for Juan's and Jihyeon's batch using the finished confirmed and denied lists 

In [15]:
JSconfirmed=pkl.loads(decompress(storage.get(datasetName2+'JSErrorConfirmed')))
JSdenied=pkl.loads(decompress(storage.get(datasetName2+'JSErrorDenied')))

In [16]:
#Juan's Error Analysis
fplist=[key for key in JuanConfirmed if key not in JSconfirmed]
tplist=[key for key in JuanConfirmed if key in JSconfirmed]
fnlist=[key for key in JuanDenied if key not in JSdenied]
tnlist=[key for key in JuanDenied if key in JSdenied]
fp=len(fplist)/(len(fplist)+len(tnlist))
tp=len(tplist)/(len(tplist)+len(fnlist))
fn=len(fnlist)/(len(fnlist)+len(tplist))
tn=len(tnlist)/(len(tnlist)+len(fplist))
p=len(tplist)/(len(tplist)+len(fplist))
r=len(tplist)/(len(tplist)+len(fnlist))
print('Error rates for Juan\'s Batch',JSbatch)
print('False Positive:',fp)
print('True Positive:',tp)
print('False Negative:',fn)
print('True Negative:',tn)
print('Precision:',p)
print('Recall:',r)

Error rates for Juan's Batch 0
False Positive: 0.03080872913992298
True Positive: 0.9847036328871893
False Negative: 0.015296367112810707
True Negative: 0.9691912708600771
Precision: 0.9147424511545293
Recall: 0.9847036328871893


In [ ]:
JSMap=wf.interactive.MapApp()
JSMap.clear_layers()
JSMap.center=(22.3094, 83.4160)
JSMap.zoom=6
def showPopup(event,feature,properties):
    JSMap.add_layer(Popup(location=(dl.scenes.DLTile.from_key(properties['style']['key']).geometry.centroid.xy[1][0],dl.scenes.DLTile.from_key(properties['style']['key']).geometry.centroid.xy[0][0]),child=HTML(value='Key: '+properties['style']['key']+'<br>Description: '+properties['style']['description']),close_button=True))

In [ ]:
#FALSE POSITIVES
for key in fplist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"red",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'False Positive'},
        hover_style={'color':'blue'},
    )
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)

In [ ]:
JSMap.clear_layers()
#FALSE NEGATIVES
for key in fnlist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"orange",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'False Negative'},
        hover_style={'color':'blue'},
    )
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)

In [ ]:
JSMap.clear_layers()
#TRUE POSITIVE
for key in tplist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"green",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'True Positive'},
        hover_style={'color':'blue'},
    )    
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)    

In [ ]:
JSMap.clear_layers()
for key in tnlist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"yellow",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'True Negative'},
        hover_style={'color':'blue'},
    )    
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)

In [ ]:
JSMap

## Next Dataset!

In [17]:
#Initialize the basic variables
storage=Storage(auth=Auth(client_id=config.ID,client_secret=config.SECRET))

#Juan's Data, can only be 0!
JSbatch=random.randint(0,0)
print('Working on ',JSbatch)

Working on  0


## This is Juan's Labeled Dataset!

In [6]:
print("Juan's Batch #",JSbatch)
if datasetName3+'JSErrorConfirmed' not in storage.list():
    storage.set(datasetName3+'JSErrorConfirmed',compress(pkl.dumps({})))
    JSconfirmed={}
else:
    JSconfirmed=pkl.loads(decompress(storage.get(datasetName3+'JSErrorConfirmed')))
if datasetName3+'JSErrorDenied' not in storage.list():
    storage.set(datasetName3+'JSErrorDenied',compress(pkl.dumps({})))
    JSdenied={}
else:    
    JSdenied=pkl.loads(decompress(storage.get(datasetName3+'JSErrorDenied')))
JSUI=InitiateUI(JSconfirmed,JSdenied,pkl.loads(decompress(storage.get(datasetName3+'GroupBatch'+str(JSbatch)))),JSbatch,datasetName3)

Juan's Batch # 0
Loading Data
Initializing UI!
Adding group # 0  to the web app
Adding group # 1  to the web app
Adding group # 2  to the web app
Adding group # 3  to the web app
Adding group # 4  to the web app
Done!


In [7]:
#Call the result to display in the below cell, continue until images no longer load.
JSUI

`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not insta…

## Create separate datasets for Juan and Jihyeon from their labeled data

In [18]:
#Get separate Batch datasets for Juan and Jihyeon!
Combinedconfirmed=pkl.loads(decompress(storage.get(datasetName3+'Confirmed'))).keys()
Combineddenied=pkl.loads(decompress(storage.get(datasetName3+'Denied'))).keys()
JuanBatch=pkl.loads(decompress(storage.get(datasetName3+'GroupBatch'+str(JSbatch))))
JuanBatchKeys=[]
for x in JuanBatch:
    JuanBatchKeys.extend(JuanBatch[x])

JuanConfirmed={key for key in JuanBatchKeys if key in Combinedconfirmed}
JuanDenied={key for key in JuanBatchKeys if key in Combineddenied}

In [19]:
#Print number to ensure all data has been loaded and separated
print(len(JuanConfirmed)+len(JuanDenied))
print(len(JuanBatchKeys))

414
414


## Calculate the Error Rates for Juan's and Jihyeon's batch using the finished confirmed and denied lists 

In [20]:
JSconfirmed=pkl.loads(decompress(storage.get(datasetName3+'JSErrorConfirmed')))
JSdenied=pkl.loads(decompress(storage.get(datasetName3+'JSErrorDenied')))

In [21]:
#Juan's Error Analysis
fplist=[key for key in JuanConfirmed if key not in JSconfirmed]
tplist=[key for key in JuanConfirmed if key in JSconfirmed]
fnlist=[key for key in JuanDenied if key not in JSdenied]
tnlist=[key for key in JuanDenied if key in JSdenied]
fp=len(fplist)/(len(fplist)+len(tnlist))
tp=len(tplist)/(len(tplist)+len(fnlist))
fn=len(fnlist)/(len(fnlist)+len(tplist))
tn=len(tnlist)/(len(tnlist)+len(fplist))
p=len(tplist)/(len(tplist)+len(fplist))
r=len(tplist)/(len(tplist)+len(fnlist))
print('Error rates for Juan\'s Batch',JSbatch)
print('False Positive:',fp)
print('True Positive:',tp)
print('False Negative:',fn)
print('True Negative:',tn)
print('Precision:',p)
print('Recall:',r)

Error rates for Juan's Batch 0
False Positive: 0.0030581039755351682
True Positive: 0.9885057471264368
False Negative: 0.011494252873563218
True Negative: 0.9969418960244648
Precision: 0.9885057471264368
Recall: 0.9885057471264368


In [ ]:
JSMap=wf.interactive.MapApp()
JSMap.clear_layers()
JSMap.center=(22.3094, 83.4160)
JSMap.zoom=6
def showPopup(event,feature,properties):
    JSMap.add_layer(Popup(location=(dl.scenes.DLTile.from_key(properties['style']['key']).geometry.centroid.xy[1][0],dl.scenes.DLTile.from_key(properties['style']['key']).geometry.centroid.xy[0][0]),child=HTML(value='Key: '+properties['style']['key']+'<br>Description: '+properties['style']['description']),close_button=True))

In [ ]:
#FALSE POSITIVES
for key in fplist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"red",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'False Positive'},
        hover_style={'color':'blue'},
    )
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)

In [ ]:
JSMap.clear_layers()
#FALSE NEGATIVES
for key in fnlist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"orange",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'False Negative'},
        hover_style={'color':'blue'},
    )
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)

In [ ]:
JSMap.clear_layers()
#TRUE POSITIVE
for key in tplist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"green",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'True Positive'},
        hover_style={'color':'blue'},
    )    
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)    

In [ ]:
JSMap.clear_layers()
for key in tnlist:
    gjs=GeoJSON(
        data=geojson.FeatureCollection([dl.scenes.DLTile.from_key(key).__geo_interface__]),
        style={"color":"yellow",
            "fillOpacity":0,
            'weight':4,
            'key':key,
            'description':'True Negative'},
        hover_style={'color':'blue'},
    )    
    gjs.on_click(showPopup)
    JSMap.add_layer(gjs)

In [ ]:
JSMap

In [ ]:
print('hi')